In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import yfinance as yf
from helper_functions import *
from datetime import date, datetime

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
import warnings
warnings.filterwarnings('ignore')

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import yfinance as yf

from sklearn.metrics import normalized_mutual_info_score
from MCMC.MCMC import MCMC

import eikon as ek
ek.set_app_key('9a249e0411184cf49e553b61a6e76c52d295ec17')

In [2]:
%load_ext autoreload
%autoreload 2

In [16]:
#tickers = niftydata()
index = ".NDX"
constituents = get_constituents(index)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all+constituents.iloc[i][0]
tickers = list(set(constituents_all))
#tickers = [ticker[:-3] for ticker in tickers ]   #use with investpy


In [67]:
recalibrating_months = 1
top_nassets = 10
training_period = 24 #24/48/96
dates_recalibrating = valid_dates(pd.date_range(start="2007-01-01", end="2024-06-15", freq=f'{recalibrating_months}M'))
data_inp = prepare_portfolio_data(tickers, recalibrating_months,"reuters")


In [68]:
assets = get_weights_stocks(constituents, top_nassets, recalibrating_months, training_period, dates_recalibrating, data_inp, save=False)

ValueError: `bins[0]` must be positive, when an integer

In [ ]:
assets

In [ ]:
with open(f'NAZDAQ_RecalibPeriod_{int(recalibrating_months)}.pkl', 'wb') as file:
    pickle.dump(assets, file)

In [ ]:
with open(f'BSE_RecalibPeriod_{int(1)}.pkl', 'rb') as file:
    assets = pickle.load(file)

In [ ]:
# for date_i in range(len(dates_recalibrating) - (int(training_period/recalibrating_months) + 1)):
#     print(f"Training period begins: {dates_recalibrating[date_i]}")
#     print(f"Training period ends: {dates_recalibrating[date_i + int(training_period/recalibrating_months)]}")
#     print(f"Data used for training between {dates_recalibrating[date_i]} and {dates_recalibrating[date_i + int(training_period/recalibrating_months)-1]}")
#     print(f"Testing period begins: {dates_recalibrating[date_i + int(training_period/recalibrating_months)]}")
#     print(f"Testing period ends: {dates_recalibrating[date_i + int(training_period/recalibrating_months) + 1]}")

In [ ]:
rebalancing_months = 12
dates_rebalancing = []
assetsb = []
for date_i in range(len(dates_recalibrating)-(int(training_period/recalibrating_months)+1)):
    if (recalibrating_months*date_i)%rebalancing_months==1:
        dates_rebalancing.append(dates_recalibrating[date_i + int(training_period/recalibrating_months)])
        assetsb.append(assets[date_i])
dates_rebalancing.append(pd.to_datetime(date.today()))

In [ ]:
for date_i in range(len(dates_rebalancing)-1):
    print("*"*30)
    print(f"Testing period begins: {dates_rebalancing[date_i]}")
    print("Selected Assets")
    print(assetsb[date_i])
    print(f"Testing period ends: {dates_rebalancing[date_i + 1]}")

In [ ]:
inputs = []
for date_i in range(len(dates_rebalancing)-1):
    inputs.append([date_i, dates_rebalancing, data_inp, assetsb])
try:
    pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
    results_backtest = pool.map(backtest_AM, inputs)
finally:  # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()

results_final = pd.DataFrame()
for tt in results_backtest:
    results_final = pd.concat([results_final, tt], axis=0)


In [ ]:
import yfinance as yf
bench = yf.download("^BSESN", start = '2011-01-31', end= str(date.today()))
bench["Return"] = np.log(bench["Close"]/bench["Close"].shift(1))

In [ ]:
# bench = pd.DataFrame()
# bench["Return"] = data_inp[[f"{ticker}Return" for ticker in tickers]].mean(skipna=True, axis=1)
# bench.set_index(data_inp["Date"], inplace=True)
temp_res = pd.concat([results_final.rename(columns={0:"S_Return"}),bench["Return"]], join="inner",axis = 1)
temp_res['Market_Return'] = temp_res['Return'].expanding().sum()
temp_res['Strategy_Return'] = temp_res['S_Return'].expanding().sum()
temp_res['Portfolio Value'] = ((temp_res['Strategy_Return'] + 1) * 10000)
temp_res = temp_res.reset_index().rename(columns={'index':"Date"})


In [ ]:
plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
plt.title('Strategy Backtest')
plt.legend(loc=0)
plt.tight_layout()
plt.show()

In [ ]:
print(f"Sortino: {backtest_sortino(temp_res,0,0)}")
print(f"Sharpe: {backtest_sharpe(temp_res,0,0)}")
print(f"Rolling Sortino: {backtest_rolling_sortino(temp_res,0,0)}")
print(f"Rolling Sharpe: {backtest_rolling_sharpe(temp_res,0,0)}")
print(f"Rolling CAGR: {backtest_rolling_cagr(temp_res,0,0)}")
print(f"Rolling MaxDrawup/MaxDrawdown: {backtest_maxdrawup_by_maxdrawdown(temp_res,0,0)}")
print(f"Rolling Outperformance: {backtest_outperformance(temp_res,0,0)}")